## Problem Set 1 Question 2

First, we load libraries again, included in e.g. Conda python. 

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (11, 5)  #set default figure size

import numpy as np

## Implement the model


In [11]:
class iam():

    def __init__(self, σ=1, β=0.75, δ=0.5, α=1/3,
                a1 = 0.06, a2 = 2.8, b1 = 0.5, b2 = 2,
                γ = 0.13,
                φ = 0.06):
        'Parameters:'
        self.σ, self.β, self.δ, self.α, self.a1, self.a2, self.b1, self.b2, self.γ, self.φ = σ, β, δ, α, a1, a2, b1, b2, γ, φ
        
        'Endogenous outcomes and endogenous grid:'
        self.k0 = 0.1
        self.m0 = 0
        
        self.steadystate = self.ss()
        self.grid = self.create_grid()
        self.v_initial = self.v_initial()

    def u(self, c):
        'Utility function'
        σ = self.σ

        return c ** (1 - σ) / (1 - σ) if σ!= 1 else np.log(c)

    def u_prime(self, c):
        'Derivative of utility'
        σ = self.σ

        return c ** (-σ)

    def f(self, k):
        'Production function'
        α = self.α

        return k ** α

    def f_prime(self, k):
        'Derivative of production function'
        α = self.α

        return α * k ** (α - 1)
    
    def f_prime_inv(self, k):
        'Inverse of derivative of production function'
        α = self.α 

        return (k / α) ** (1 / (α - 1))

    def ss(self):
        'Define the steady-state equation s.t. RHS=0'
        β, δ = self.β, self.δ
        f_prime_inv = self.f_prime_inv
        
        return f_prime_inv(1/β - (1-δ))
    
    def create_grid(self):
        'Create a grid as prescribed in the assignment'
        n_k = 100j
        n_m = 100j
        k1, k_nk = 0.1, 0.4
        m1, m_nm = 0, 0.5

        s = np.mgrid[k1:k_nk:n_k, m1:m_nm:n_m].reshape(2,-1).T

        return s
    
    def v_initial(self):
        'Create initialization for value function'
        grid = self.grid
        v_initial = np.zeros_like(grid)
        return v_initial      
    
    def state_action_value(self, k_new, m_new, k, m, v_array):
        # interpolation of value function (probably not necessary)
        v = lambda x: interp(self.grid, v_array, x)
        #functions
        u, f = self.u, self.f
        #parameters
        β, δ, a1, a2, b1, b2, φ, γ, α = self.β, self.δ, self.a1, self.a2, self.b1, self.b2, self.φ, self.γ, self.α
        
        #write the bellman eq. in separate, tractable parts
        first_part = ((m_new - (1-φ)*m)*k**(-α))/γ
        second_part = (1-a1*(1-first_part)**(a2))*k**α
        third_part = second_part / (1 + m**b2)
        final_part = third_part + (1-δ)*k - k_new
                
        return  u(final_part) + β*v(k_new, m_new)   #k_new is the thing we solve for
                                                  

## Implement the algorithm

In [ ]:
# Edit this - restrictions on c(t) and mu(t) in the matrix
def u_mat(ngm, v):
    grid = ngm.grid
    u, f = ngm.u, ngm.f
    β, δ = ngm.β, ngm.δ
    
    u_out = np.zeros((len(ngm.grid), len(ngm.grid)))
    
    for i in range(len(grid)):
        
        for j in range(len(grid)):
            
            if (grid[j] > (1-δ)*grid[i] and grid[j] < f(grid[i]) + (1-δ)*grid[i]):
                
                u_out[i,j] = u(f(grid[i]) + (1-δ)*grid[i] - grid[j]) + β*v[j]
            
            else:
                u_out[i,j] = -np.Inf
                   
    return u_out